# Lanzar el cluster con todo instalado

## Instalar librerias y complementos

In [ ]:
! pip install pandas
! pip install pyspark
! pip install metapy
! pip install boto3

# Cargar librerias

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
import metapy
import requests, zipfile, io, os, boto3

# Cargar datos necesarios 

In [24]:
s3 =  boto3.client('s3', region_name='us-east-1')
with open('data/news/news.dat', 'wb') as f:
    s3.download_fileobj('finaltext','news.dat', f)

obj = s3.get_object(Bucket='finaltext', Key=u'news.csv')
df = pd.read_csv(obj['Body'])


## Inverted index using metapy


In [25]:
#!rm -rf news-idx
idx = metapy.index.make_inverted_index('miniconfig.toml')


# IR: Querys

In [33]:
ranker = metapy.index.OkapiBM25()
query = metapy.index.Document()
query.content('Trump hates china') # query from AP news
top_docs = ranker.score(idx, query, num_results=5)

index=[tup[0] for tup in top_docs]
df.loc[index,['title','content']]


,title,content
23373,Ann Coulter: I Might Have Been Killed at the B...,"Ann Coulter, in a major interview with Vanity ..."
55967,A Trump Supporter Dwells in Beijing,BEIJING — Ardent Chinese supporters of Donald...
123632,"For Chinese officials, Trump perhaps better th...","In 2010, then Secretary of State Hillary Clin..."
119243,Massachusetts college apologizes for racist tw...,Salem State University President Patricia Mese...
25871,Donald Trump’s Hypocrisies - Breitbart,Part of Donald Trump’s appeal as a candidate i...
